In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
data=pd.read_csv('dataset.csv')

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
features = ['ID', 'society', 'location', 'price', 'total_sqft', 'surface_area', 'bedroom', 'bathroom', 'carport']

In [ ]:
numeric_features = ['price', 'total_sqft', 'surface_area', 'bedroom', 'bathroom', 'carport']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
nlp_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('nlp', nlp_transformer, ['location']),
        ('num', numeric_transformer, numeric_features)
    ]
)

In [ ]:
search_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('search', NearestNeighbors(n_neighbors=3, algorithm='ball_tree'))
])

In [ ]:
search_model.fit(train_data[features])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nlp',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['location']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['price', 'total_sqft',
                                                   'surface_area', 'bedroom',
                                                   'bathroom', 'carport'])])),
                ('search',
                 NearestNeighbors(algorithm='ball_tree', n_neighbors=3))])

In [ ]:
def search_houses(query):
    # Transformasi query menggunakan preprocessor
    query_transformed = preprocessor.transform(pd.DataFrame([query], columns=features))
    
    # Temukan rumah terdekat berdasarkan query
    _, indices = search_model.named_steps['search'].kneighbors(query_transformed)

    # Ambil data rumah terdekat dari data latih
    result_houses = train_data.iloc[indices[0]][features]

    return result_houses

In [ ]:
query = {'location': 'Babelan', 'price': 480000000, 'total_sqft': 36, 'surface_area': 60}
result = search_houses(query)

In [ ]:
print("Hasil Pencarian:")
print(result)

Hasil Pencarian:
      ID                   society location      price  total_sqft  \
315  316  BUMI PANDAWA SEJAHTERA 2  Babelan  390000000          36   
200  201       KOTA BABELAN PERMAI  Babelan  390000000          36   
301  302      ALAM SAMUDRA INDAH 3  Babelan  385000000          36   

     surface_area  bedroom  bathroom  carport  
315            60        2         1        1  
200            60        2         1        1  
301            60        2         1        1  
